In [ ]:
%pylab inline
plt.style.use('seaborn-talk')

We will use this function to plot the metrics of our training process, to help in choosing the best model

In [ ]:
def plot_metric(history, metric):
    history_dict = history.history
    values = history_dict[metric]
    if 'val_' + metric in history_dict.keys():  
        val_values = history_dict['val_' + metric]

    epochs = range(1, len(values) + 1)

    if 'val_' + metric in history_dict.keys():  
        plt.plot(epochs, val_values, label='Validation')
    plt.semilogy(epochs, values, label='Training')

    if 'val_' + metric in history_dict.keys():  
        plt.title('Training and validation %s' % metric)
    else:
        plt.title('Training %s' % metric)
    plt.xlabel('Epochs')
    plt.ylabel(metric.capitalize())
    plt.legend()
    plt.grid()

    plt.show()  

We will use this function to explore the data

In [ ]:
def plot_mnist_image(N, imgs, labels):
    print("The image below should show the number %d" % labels[N])
    plt.imshow(imgs[N,], cmap=plt.cm.binary)
    plt.grid(True)

# The problem

We will recognize handwritten digits. For an image containing a number between 0 and 9, we will recognize the number and will produce an int as output

In [ ]:
from keras.datasets import mnist

Load minst images using the method `load_data()`. Be carefull, as it returns a tuple with the image and the lables

## A note on the curse of dimensionality

Define a similarity measure. For instance, the [cosine similarity](https://en.wikipedia.org/wiki/Cosine_similarity), that is 1 if vectors are parallel (most similar), and zero if they are perpendicular (lack of correlation).

In [ ]:
from scipy import spatial

def cosine_similarity(img1, img2):
    # reshape input matrices as vectors
    img1_ = img1.reshape((28*28,))
    img2_ = img2.reshape((28*28,))
    # normalize input vectors
    norm1 = np.sqrt(np.sum(img1_*img1_))
    norm2 = np.sqrt(np.sum(img2_*img2_))
    img1_ = img1_ / norm1
    img2_ = img2_ / norm2
    sim = 1 - spatial.distance.cosine(img1_, img2_)    
    return sim

Check the similarity among two random images, and plot them. Does it make sense?

What about the similarity among images of numbers 5?

Try the similarity between 5 and 4, or 5 and 8, for instance

<div class="alert alert-success">
As the dimension of our input increases, everything starts looking the same. Distances in high dimensions cannot be used! This is a consequence of the curse of dimensionality.
</div>

We will design a network that will be able to recognize the number shown at the image (for all the images in the test set)

## Data transform

We need to change the shape of the data, so it can be fed to the network more easily

In [ ]:
# get the min, max, meand and std of training images


All the images are 28x28 matrices, with values between 0 and 255. Let's normalize the images, to avoid problems in the numerical computations using large numbers.

We will convert the matrix to a vector with 28x28 components, stacking columns on top of each other. This will make the design of the network.
* Alternatively, we could add a `Flatten(input_shape=(28,28))` layer to the model

In [ ]:
# normalize image's values to the maximum


Because we want to assign a *category* to each image, we need to transform the output to categorical format

In [ ]:
from keras.utils import to_categorical

Initially, our target data is just a set of numbers

We are going to transform it to **1-HOT encoding format**

In [ ]:
train_labels_t = to_categorical(<fill in>)

We have to repeat the same process with the test data

## Let's build the model

In [ ]:
from keras import models
from keras import layers

In [ ]:
def build_model():
    <fill in>

Here we will make some decisions about how to train our model.

The **objective function**, called also the **loss function** in Machine Learning, will be *categorical crossentropy*. This is because we are trying to predict discrete classes. If we choose a different function, the solution will be different (take a look at [Keras available losses](https://keras.io/losses/)). This is the function that will tell us when we have found the *solution*. Different functions will point to different *solutions*. The categorical crossentropy for a single observation (exmaple in train, for instance) is defined as:

$$-\sum_{{\rm category} = 1}^M y_{\rm category} \log(\hat{y}_{\rm category}).$$

Have a look at this [cheatsheet](https://ml-cheatsheet.readthedocs.io/en/latest/loss_functions.html).

The **optimizer** is the method that we will use to find the minimum of the **loss function**. In general, we will not find the global minimum of the loss function, but a minimum that is good enough. *RMSProp* is one the variants of gradient descent. But there are many more [availbale in Keras](https://keras.io/optimizers/). We will deep dive into the optimization problem later in this course. 
 
These two settings will define the family of solutions we will find during the training process.

The other parameter, the **metrics**, is only for information purposes. In each step of the training (called **epoch**), Keras will report the value of the metrics. But whether we choose one metric or another will not influence the training process. It is only for reporting how the training is going. That information will be useful for the validation of the model, that is, for the **hyperparameters tuning**.


In [ ]:
from keras import optimizers
from keras import losses
from keras import metrics

In [ ]:
m = build_model()

In [ ]:
# compile the model


In [ ]:
# get the summary
m.summary()

Now let's fit the neural network. We will keep the results in a *history* object to plot some parameters after the training, and use that information for improving our model (changing layers, activation functions, etc). This process is called **hyperparameters tuning** or hypeparameters search.

Here we need to set two parameters: the number of **epochs** and the **batch size**.

In previous cells, I have said that the training process is an optimization problem: we try to find the minimum.

That's actually not entirely true. In practice, we don't keep searching for a minimum. Instead, we train for a preset number of epochs. Then we analyze the output using the validation set. If we detect overfit, then we reduce the number of epochs. If we don't detect it, then maybe we can keep going for some more epochs to find a better model. The number of epochs is another hyperparameter. In general, we should stop the training as early as possible. Any further epoch can make our model overfit, and therefore generalize poorly.

The batch size is another hyperparameter. The network is not updated item by item, vector by vector. We can actually calculate the weights for a batch of several items, forming a matrix or a tensor. The batch size will affect the granularity of the calculations and the performance. A larger batch size will probably result in a slightly worse accuracy, but in a better performance. As long as the batch size is not so huge that it cannot be kept in memory. Because we are running on a GPU, it is a good idea to use a power of 2 for the batch size. It will probably make a better use of the GPU. But again, the effect of the batch size must be tested using the validation set, and adjusted until we find a good value for that parameter.

An epoch is a full pass over the training dataset. At the end of each epoch, Keras will calculate the value of the metrics for the training set, and the loss function and metrics for the validation set. The loss and metrics of the validation set are useful for the hyperparameters tuning process.



In [ ]:
# train!
h = <fill in>

In [ ]:
plot_metric(h, 'loss')

In [ ]:
plot_metric(h, 'categorical_accuracy')


Now let's check how well the model works with the test dataset. Remember that we don't have used it at all during the training and tuning of the model.

## Analyze the classification performance

For this, we will use the model's `evaluation` method. 

In [ ]:
loss, acc = m.evaluate(<test images>, <test labels>)

In [ ]:
loss, acc

Let's see how well it predicts some random items from the test set.

In [ ]:
# plot a random image 


The prediction is a 10-element vector (*1-HOT encoded vector*), with (something like) the probability of each class. The location of the max is giving us the predicted class.

In [ ]:
# get the prediction of that same image. This process is what it's called **inference**.


In [ ]:
np.argmax(<your prediciton>)

## Exercises

**EXERCISE 1**: Change the [activation function](https://keras.io/activations/) in the model. How does it affect accuracy? What happens if you use linear activation function? What happens if we use the $\tanh$ function? Does it affect to overfitting?

 
Here we see that the accuracy keeps increasing over epochs. An overfitted model would produce a decrease of the validation accuracy at some point. This means that we have found an optimal model. The validation loss is stuck at a minimum, but the accuracy is not affected by the extra epochs.

_**EXERCISE 2**_: How is the model accuracy affected if we start over and fit for 40 epochs? And for 4? And for 400?

_**EXERCISE 4**_: What is the most simple model that you can get that achieves a similar validation performance (accuracy)? (Note that the loss values might not be comparable if we use regularization techniques, change the network architecture, etc.)

_**EXERCISE 5**_: Can you find a test item that is predicted wrongly? How many images are predicted wrongly? Can you find all the items that are wrongly classified? 

_**EXERCISE 6**_: Some digits are more difficult to recognize than others. Because we know the test labels, we can find out how many times the corresponding test images are misclassified. Could you find what are the top 3 test labels that are more often misclassified?